In [1]:
import accounts, banks, requests, json, pandas as pd, tqdm
from obp_client import client
import obp_python as obp

In [ ]:
bank_cols = ['bank_id','bank_code','full_name','logo','website','bank_robuting_scheme','bank_routing_address','list_of_attribute_names_and_values']
account_cols = ['bank_id','product_code','sole_customer_number','joint_customer_numbers','account_status','issue_date','last_movement_date','balance','currency','label','bank_routing_scheme','bank_routing_address','account_routing_scheme','account_routing_address','branch_routing_scheme','branch_routing_address','account_attributes_list']
transaction_cols = ['transaction_id','initiating_party','debtor_legal_name','debtor_bank_routing_scheme','debtor_bank_routing_address','debtor_account_routing_scheme','debtor_account_routing_address','creditor_legal_name','creditor_bank_routing_scheme','creditor_bank_routing_address','creditor_account_routing_scheme','creditor_account_routing_address','amount','currency','description','booked_date','value_date','transaction_attributes_list']
customer_cols = ['bank_id','customer_number','legal_name','mobile_phone_number','email','date_of_birth','title','branch_id','name_suffix','customer_attributes_list']

In [2]:
bank_id = "d8839721-ad8f-45dd-9f78-2080414b9xf9"

In [ ]:
#Grant User All Entitlements
my_userid = "aa1908be-8a4f-4ef2-8373-54b9cd46f50f"
role_api = obp.RoleApi(client)
entitlement_api = obp.EntitlementApi(client)
for role in role_api.get_roles({}).to_dict()['roles']:
    body = {'role_name': role['role']}
    if role['requires_bank_id']:
        body.update({"bank_id": bank_id})
    try:
        entitlement_api.add_entitlement(user_id=my_userid, body=body)
    except:
        pass

In [3]:
bank_id = "d8839721-ad8f-45dd-9f78-2080414b9xf9"
banks.banks.get_bank(bank_id=bank_id)

{'attributes': [],
 'bank_code': 'OBPJ',
 'bank_routings': [{'address': 'd8839721-ad8f-45dd-9f78-2080414b9xf9',
                    'scheme': 'OBP'},
                   {'address': 'd8839721-ad8f-45dd-9f78-2080414b9xf9',
                    'scheme': '000000000'}],
 'full_name': 'Open Bank Project Jordan',
 'id': 'd8839721-ad8f-45dd-9f78-2080414b9xf9',
 'logo': 'https://static.openbankproject.com/jopacc/img/joinfincubator.png',
 'website': 'www.openbankproject.com'}

In [4]:
loan_data = pd.read_csv("../Data/LoanData.csv")

C:\Users\Fadi-T480\AppData\Local\Temp\ipykernel_11008\386382346.py:1: DtypeWarning: Columns (12,34,38,80,82,83,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv("../Data/LoanData.csv")


In [5]:
occupation_Area = {0:"Unknown" ,1: "Other", 2: "Mining", 3: "Processing", 4: "Energy", 5: "Utilities", 6: "Construction", 7: "Retail\nand\nwholesale", 8: "Transport\nand\nwarehousing", 9: "Hospitality\nand\ncatering", 10: "Info\ntelecom", 11: "Finance\ninsurance", 12: "Real-estate", 13: "Research", 14: "Administrative", 15: "Civil service", 16: "Education", 17: "Healthcare", 18: "Entertainment", 19: "Agriculture"}
employment_status = {-1: 'Unknown',0: 'Unknown', 1: 'Unemployed', 2: 'Partially employed', 3: 'Fully employed', 4: 'Self-employed', 5: 'Entrepreneur', 6: 'Retiree'}
country = {"EE": 'Estonia', 'FI': 'Finland', 'ES': 'Spain', 'SK': 'Slovakia', 'NL': 'Netherlands'}
gender = {0:'Male', 1: 'Female', 2: 'Unknown'}
education = {0: 'Unknown', 1:'Primary education', 2: 'Basic education', 3: 'Vocational education', 4: 'Secondary education', 5: 'Higher education'}
marital_status = {0: 'Unknown', 1: 'Married', 2: 'Cohabitant', 3: 'Single', 4: 'Divorced', 5: 'Widow'}
home_ownership = {0: 'Homeless', 1: 'Owner', 2: 'Living with parents', 3: 'Tenant pre-furnished property', 4: 'Tenant unfurnished property', 5: 'Council house', 6: 'Joint tenant', 7: 'Joint ownership', 8: 'Mortgage', 9: 'Owner with encumbrance', 10: 'Other'}
features = ['Age','Gender' ,'Education','MaritalStatus','NrOfDependants','EmploymentStatus','EmploymentDurationCurrentEmployer','WorkExperience','OccupationArea','HomeOwnershipType','IncomeTotal', 'LiabilitiesTotal','FreeCash']

translator = {'OccupationArea': occupation_Area, 'EmploymentStatus': employment_status, 'Country': country, 'Gender': gender}
translator.update({'Education': education, 'MaritalStatus': marital_status, 'HomeOwnershipType': home_ownership})

In [8]:
number_of_requests = (len(loan_data)*len(features))+len(loan_data)
pbar = tqdm.tqdm(total=number_of_requests)

  0%|                                                                                                                          | 0/3879904 [00:00<?, ?it/s]

In [ ]:
#Import Loan Account data + Attributes to OBP
loan_data = loan_data.replace(translator)
for i in loan_data.index:
    row = loan_data.iloc[i]
    account_id = row['LoanId']
    account_data = {"user_id":"aa1908be-8a4f-4ef2-8373-54b9cd46f50f",  "label": account_id,  "product_code":"Housing Loan",  "balance":{    "currency":"EUR",    "amount":"0"  },  "branch_id":"DERBY6",  "account_routings":[{    "scheme":"AccountNumber",    "address": account_id  }]}
    try:
        accounts.accounts.create_account(bank_id=bank_id, account_id=account_id, body=account_data)
        pbar.update(1)
    except:
        pass
    attribute_list = row[features].to_dict()
    for k,v in attribute_list.items():
        body = {"name": k,  "type":"STRING",  "value": v,  "product_instance_code":"Housing Loan"}
        try:
            accounts.accounts.create_account_attribute(bank_id=bank_id, account_id=account_id, product_code="Housing Loan", body=body)
            pbar.update(1)
        except:
            pass

  0%|▏                                                                                                          | 8304/3879904 [16:21<126:23:49,  8.51it/s]2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/product

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

...

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C72FA9C280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /obp/v5.0.0/banks/d8839721-ad8f-45dd-9f78-2080414b9xf9/accounts/34045374-634C-4E25-B141-2E865CA030E1/products/Housing%20Loan/attribute
2023-03-17 18:11:58,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection 

In [ ]:
accounts.accounts.get_accounts_held(body={}, bank_id=bank_id)

In [ ]:
# Define Housing Loan as Product
product_body = {  "parent_product_code":"",  "name":"Housing Loan 1",  "more_info_url":"www.moontestingbank.com/housingloan",  "terms_and_conditions_url":"www.example.com/xyz",  "description":"Housing Loan for local residents",  "meta":{    "license":{      "id":"ODbL-1.0",      "name":"Open Database License"    }  }}
#obp.api.ProductApi(client).create_product(bank_id=bank_id, product_code="Housing Loan", body=product_body)
print(obp.api.ProductApi(client).get_products(bank_id=bank_id))